# IPython EUPS magics

### What

This IPython extension adds a new `%eups` magic that allows one to call EUPS from IPython, and (importantly) setup products into the current IPython notebook's environment.

This comes in very handy when we want the Python notebook to fully capture the products (possibly down to the product version) that were used to execute it.

It's also handy when in the middle of some analysis you realize you've forgotten to setup a product you need.

### Installing

#### From EUPS package

This extension is available as an EUPS package at EUPSForge:

    eups distrib -r http://eupsforge.net ipython_eups
    setup ipython_eups

Once you've installed it, you may want to automatically load it every time you set up your EUPS environment. E.g., if you're using `loadLSST.bash` (or any of the `loadLSST.*` scripts), just edit it and append:

    setup ipython_eups

to the end.

#### From source

The sources are on github, and running from source is pretty straightforward as well:

    git clone http://github.com/mjuric/ipython_eups
    setup -r ipython_eups


## The basics

To activate the `%eups` magic make sure you've `setup`-ed the `ipython_eups` module and then run:

In [1]:
%load_ext eups_magic

The `%eups` magic lets you run any EUPS subcommand. For example, let's see which products are available:

In [2]:
%eups list

ipython_eups          LOCAL:/Users/mjuric/projects/incubator/eups-ipython-magic 	setup
oorb                  lsst-g650e0a6f6c


Let's try to import a module that hasn't been set up.

In [3]:
import pyoorb

ImportError: No module named pyoorb

OK, so we'll set it up first:

In [4]:
%eups setup oorb lsst-g650e0a6f6c

Let's check if this worked:

In [5]:
%eups list -s

ipython_eups          LOCAL:/Users/mjuric/projects/incubator/eups-ipython-magic 	setup
oorb                  lsst-g650e0a6f6c 	setup


... and now we can use its modules directly from this notebook:

In [6]:
import pyoorb

We can also unsetup a module.

Note however that because of how Python handles modules, this will not unload an already loaded module. Furthermore, while pure-Python modules can be reloaded with reload(), that is not possible for C extensions (see http://bugs.python.org/issue1144263).

Bottom line: this will not enable you to switch out a version of a module at runtime, unless the module is pure Python and you remember to call reload.

In [7]:
%eups unsetup oorb lsst-g650e0a6f6c

In [8]:
%eups list -s

ipython_eups          LOCAL:/Users/mjuric/projects/incubator/eups-ipython-magic 	setup


## Miscellaneous

Errors get appropriately highlighted:

In [9]:
%eups foobar

ERROR: Command 'eups foobar ' returned non-zero exit status 10
eups foobar: Unrecognized command: foobar



We can setting up a list of products in a loop, using IPython variable expansion:

In [10]:
products = [('oorb', 'lsst-g650e0a6f6c')]
for product, version in products:
    %eups setup $product $version
%eups list -s

ipython_eups          LOCAL:/Users/mjuric/projects/incubator/eups-ipython-magic 	setup
oorb                  lsst-g650e0a6f6c 	setup


All EUPS subcommands work:

In [11]:
%eups distrib list -s http://lsst-web.ncsa.illinois.edu/~mjuric/eupsforge/

  anaconda             generic    2.1.0-lsst1
  anaconda             generic    2.1.0-4-g35ca374
  ann                  generic    master-g241439a636
  eupsforge            generic    master-g9ae1b6a1f0
  eupsforge_distrib    generic    master-g0740fbfb19
  eupsforge_distrib    generic    master-g0740fbfb19+3
  eupsforge_distrib    generic    master-g0740fbfb19+4
  eupsforge_distrib    generic    master-g4bfbf8a310
  eupsforge_distrib    generic    master-g5126290247+1
  eupsforge_distrib    generic    master-g5126290247+3
  eupsforge_distrib    generic    master-g704909cfe0
  eupsforge_distrib    generic    master-gb7d5b0be39
  firefly              generic    tickets.DM-2731-eups-for-firefly-g9c17782531
  firefly              generic    tickets.DM-2731-eups-for-firefly-geffca691b1
  git                  generic    1.8.5.2
  git                  generic    2.4.0
  git                  generic    2.4.0-1-g2a468e1
  gradle               generic    master-g72e5c9f5d6
  jdk                

## Internals

In [14]:
import os
linkfarm = os.environ['IPYTHON_EUPS_LIB_LINKFARM']
print os.environ['DYLD_LIBRARY_PATH']
print linkfarm
! ls -l $linkfarm
! ls -l $linkfarm/lib/

/Users/mjuric/projects/eups/stack/DarwinX86/oorb/lsst-g650e0a6f6c/lib:/var/folders/fn/xlbw59n15rg8ywvrxdy9xfm00000gn/T/ipython-eups-linkfarm.XXXXX.k5zTDU1D/lib
/var/folders/fn/xlbw59n15rg8ywvrxdy9xfm00000gn/T/ipython-eups-linkfarm.XXXXX.k5zTDU1D
total 8
drwx------  4 mjuric  staff  136 Jun 14 00:48 ipython-eups-linkfarm.34444.LuxNad
lrwxr-xr-x  1 mjuric  staff   34 Jun 14 00:48 lib -> ipython-eups-linkfarm.34444.LuxNad
total 16
lrwxr-xr-x  1 mjuric  staff  78 Jun 14 00:48 Makefile -> /Users/mjuric/projects/eups/stack/DarwinX86/oorb/lsst-g650e0a6f6c/lib/Makefile
lrwxr-xr-x  1 mjuric  staff  83 Jun 14 00:48 liboorb.dylib -> /Users/mjuric/projects/eups/stack/DarwinX86/oorb/lsst-g650e0a6f6c/lib/liboorb.dylib
